The Hamiltonian for the model is

$$
H = - \sum_{<ij>} J_{ij} s_i s_j - \sum_{i} B_i s_i
$$

$\sum_{<ij>}$ indicates a summation over all neighboring pairs of spins (concrete example next),
$J_{ij}$ is the strenght of the interaction between the pair $ij$,
$B_i$ is the external magnetic field at site $i$.

And for our purposes, we will work in a world where there is no external magnetic field,
and where the coupling between neighbroing pairs of spins if constant.

$$
H = - J \sum_{<ij>} s_i s_j
$$

Now let us show how to compute the energy.
We will be doing it by computing the energy between transitions (we are doing it this way because we just happen to know that this will come in handy later on).


If a sping is up, $u$, then its value is +1.
So here below we have 4 pairs of nearest neighboards where $\sum_{<ij>} s_i s_j = 1 + 1 + 1 + 1 = 4$, so $H = -J4$.
From there try double checking the rest.

```
              u                      u
E = -4J     u u u    ->   E = 4J   u d u
              u                      u
```
$\Delta E = 4J - -4J = 8J$ and so $-\beta \Delta E = - 8J \beta$


```
              u                        u
E = -2J     d u u     ->    E = 2J   d d u
              u                        u
```
$\Delta E = 2J - -2J = 4J$ and so  $-\beta \Delta E = - 4J \beta$.


```
              u                        u
E =  0J     d u u     ->    E = 0J   d d u
              d                        d
```
$\Delta E = 0J - 0J = 0J$ and so $-\beta \Delta E = - 0J \beta = 0$.


```
              d                         d
E = 2J      d u u     ->    E = -2J   d d u
              d                         d
```
$\Delta E = -2J - 2J = -4J$ and so $-\beta \Delta E = 4J \beta$.

```
              d                        d
E = 4J      d u d     ->   E = -4J   d d d
              d                        d
```
$\Delta E = -4J - 4J = -8J$ and so $-\beta \Delta E = 8J \beta$.

If you rack your brain you'll see that these are all the possible states and transitions, when we flip one spin at a time.

The critical temperature for the 2D model has been analytically obtained and is

$$
T_c = \frac{2J}{\log(1 + \sqrt{2})} \approx 2.269 J
$$

or
$$
\beta_c \approx \frac{0.4407}{J}
$$

See [Wikipedia: Onsager's exact solution](https://en.wikipedia.org/wiki/Ising_model#Onsager's_exact_solution).

# Metropolis Algorithm

Single site is updated at a time:
    a. Probability of generating such as state is $1/N$.

The detailed condition for Markov Chains Monte Carlos is
$$
\frac{P(\mu \rightarrow \nu)}{P(\nu \rightarrow \mu)}
= \frac{g(\mu \rightarrow \nu) A(\mu \rightarrow \nu)}{g(\nu \rightarrow \mu) A(\mu \rightarrow \nu)}
= \frac{A(\mu \rightarrow \nu)}{A(\mu \rightarrow \nu)}
$$

Where $g$ is the probability of generating a given state and $A$ is the probability of accepting the transition.

Remember that the more stringent expression of detailed balance to guarantee static equilibirum is written as
$$
p_mu P(\mu\rightarrow\nu) = p_nu P(\nu\rightarrow\mu)
$$

and since the probabilities $p_mu$ and $p_nu$ are $e^{-\beta E_mu}/Z$ and $e^{-\beta E_nu}/Z$ respectively,

$$
\frac{P(\mu \rightarrow \nu)}{P(\nu \rightarrow \mu)}
= \frac{A(\mu \rightarrow \nu)}{A(\mu \rightarrow \nu)}
= \frac{p_\nu}{p_\mu}
= e^{-\beta (E_\nu - E_\mu)}
$$

Note that if $E_\nu \leq E_\mu$, then $e^{-\beta (E_\nu - E_\mu)}$, then the argument of the exponent will be zero or positive, so $e^{+(\ldots)} \geq 1$.
And if $E_\nu > E_\mu$, then $e^{-\beta (E_\nu - E_\mu)}$ will looke like $e^{-(\ldots)} < 1$.
Which intuitevely makes sense, transitioning to a state $\nu$ from a state $\mu$ is favored when the end state $\nu$ has a lower energy (rememeber that the exponents are proportional to the probability of the given state).

Thus, the Metropolis algorithm 
$$
A(\mu \rightarrow \nu)
=
\begin{cases}
    1                                                    & \text{if } E_\nu \leq E_\mu \quad(\Delta E \leq 0) \\
    e^{-\beta (E_\nu - E_\mu)} = e^{-\beta \Delta E} \   & \text{if } E_\nu > E_\mu \quad(\Delta E > 0)
\end{cases}
$$

**Note:** there is a potential for a simplification here! if you go back to the beginning where we documented all the 5 possible transitions, only 2 of them meet the criteria of $E_\nu > E_\mu$ ($\Delta E > 0$).
So we only really need to compute two exponentials $e^{-4J\beta}$ and $e^{-8J\beta}$, so we could pre-compute them instead of calling a special function.
And we do do this in `Lattice.cpp` in the `Lattice(unsigned int x, unsigned int y, unsigned int RNSeed)` constructor.

# Wolff Cluster Algorithm

## Swendsen-Wang Algorithm

1. The entire lattice is partitioned into non-overlapping clusters. Each cluster contains a region where all spins are aligned in the same direction.
2. Each cluster is flipped with probability 1/2.

## Wolff Cluster Algorithm

Create only a single cluster of aligned spins that does not include the entire lattice most of the time.

1. **Randomly** select an initial seed spin.
2. The initial seed spin forms the kernel for the cluster.
3. **Probabilistically** add to the cluster all of the seed's neighbours that have the same spin.
4. Recursively consider all of the neighbours of the seed's neighbours.
5. Stop when all the properly-aligned neighbour sites have been tested.
6. Flip the cluster to create a new state. This new state is always accepted.

The Wolff cluster algorithms then moves the issue of optimizing the acceptance probabilities to
the generation of an acceptance probability (by probabilistically adding sites to a cluster).

So in this case $A(\mu\rightarrow\nu) = A(\nu\rightarrow\mu) = 1$, and we know we want to land in
$$
\frac{P(\mu \rightarrow \nu)}{P(\nu \rightarrow \mu)}
= \frac{g(\mu \rightarrow \nu)}{g(\mu \rightarrow \nu)}
= e^{-\beta (E_\nu - E_\mu)}
$$

With the Wolff algorithmn, the non-local change in energy is when the cluster is flipped and the spins at the borders of the cluster that used to be aligned end up pointing in the opposite direction.
Then, if $P_+$ is the probability of a neighbouring spin being added to a cluster, the make the argument that the selection probability $g$ is proportional to the number of independent Bernoulli trials of a neighbouring spin not being added

$$
g(\mu\rightarrow\nu) \propto (1-P_+)^n
$$

So,

$$
\frac{P(\mu \rightarrow \nu)}{P(\nu \rightarrow \mu)}
= \frac{g(\mu \rightarrow \nu)}{g(\mu \rightarrow \nu)}
= (1-P_+)^{n-m}
= e^{-\beta (E_\nu - E_\mu)}
$$

where $m$ is are the neighbours around the edges of the cluster that were pointing in the opposite direction but after the flip end up being aligned.
So going backwards (flipping the cluster back) is $g(\nu\rightarrow\mu) \propto (1-P_+)^m$.

The other thing to take into account is that if we go from $u u \rightarrow u d$, then the energy went from $-J(1) \rightarrow -J(-1)$, same if we went from $d d \rightarrow u d$. In both cases the change in energy is of $J + J = 2J$.
So the energy rises by $2J$ for each $n$ spin that used to be aligned but it no longer is after the cluster flip.
Thus,

$$
(1-P_+)^{n-m}
= (e^{-2J\beta})^{n-m}
= e^{-\beta (E_\nu - E_\mu)}
$$

Which coincidentally also tells us that,
$$
P_+ = 1 - e^{-2J\beta}
$$

# Autocorrelation Time

$\tau$ is informally, the similarity between observations of a random variable (the "state" of our system) as a function of the time lag between them.
It is roughly how long it takes for a system to evolve into a new state which is statistically independent from the initial state.

Let's say that we have some observable $Q$ (a mathematician/statistician will talk about random variables).
And let's introduce the **time-displaced autocorrelation function** $\chi(t)$,

$$
\chi(t)
= \frac{1}{t_{max} - t} \left( \sum^{t_{max}-t}_{t^\prime = 0} Q(t^\prime) Q(t+t^\prime) \right) - \langle Q \rangle^2
$$

Note that we have a sum because we are working with time series measurements of our random variables.
Also see that $t_{max}$ is the total number of measurements made during a simulation.

We assume that the correlation, and consequently $\chi(t)$,  decreases exponentially as time passes by.
So we assume a generic form such as $\chi(t) = A e^{-t/\tau}$.
Thus,
$$
\frac{\chi(t)}{\chi(0)} = e^{-t/\tau}
$$

So solving for $\tau$ gives us,
$$
\tau = -t / \log\left( \frac{\chi(t)}{\chi(0)} \right)
$$
or
$$
\tau = t / \log\left( \frac{\chi(0)}{\chi(t)} \right)
$$

To compute $\tau$ then, we would look at the ratio $\chi(t) / \chi(0)$ in the domain until $\chi(t)$ deviates from an exponential decay curve, $\chi(t) < \chi(t+1)$, or when $\chi(t) , 0$.
Within the domain in-scope then compute $\tau_t$ and then take the average.

In [3]:
# Figure 5.1 is the measurement of E vs. T on a 32x32 lattice.
# The range os Temperatures tested is from a little above 0 to 5.
# Figure 5.2 is Magnetization vs. T. With the same ranges as above.
# Figure 5.3 is the same but of specific heat vs. T.
# Figure 5.4 is susceptibility vs. T.

# Figure 5.5 is a 128x129 lattice.

# Figure 5.8 and 5.9 visualize susceptibility for lattices of size 32^2, 64^2, 128^2, 256^2, 512^2, 1024^2.